# Overview

1. [Motivation](#motivation)
1. [User benefit](#benefit)
1. [Proposed structure](#structure)
1. [Implementation](#implementation)
    1. [The `Ansatz` object](#ansatz)
    1. [Variational forms](#varforms)
    1. [Feature maps](#featmaps)
    1. Others

# Motivation<a id="motivation"></a>

## Creating a new variational form

Currently, creating a new variational form class requires a lot of copy+paste. 

Remember: `RYRZ` has all of that, too!

In [ ]:
class RY(VariationalForm):
    """Layers of Y rotations followed by entangling gates."""

    def __init__(self, num_qubits, depth=3, entangler_map=None, entanglement='full', initial_state=None,
                 entanglement_gate='cz', skip_unentangled_qubits=False, skip_final_ry=False):
        self.validate(locals())
        super().__init__()
        """ 30 more lines ... """
        self._bounds = [(-np.pi, np.pi)] * self._num_parameters
        self._support_parameterized_circuit = True

    def construct_circuit(self, parameters, q=None):
        if len(parameters) != self._num_parameters:
            raise ValueError('The number of parameters has to be {}'.format(self._num_parameters))
        """ 39 more lines ... """
        return circuit

Setting up a new variational form should be quick and easy, more like:

In [ ]:
class RY(TwoQubitAnsatz):
    def __init__(self, ...):
        super.__init__(single_qubit_gate='ry', ...)

If a user wants to quickly try a new format, it would convenient to set up the variational form in just a single line. Not all ideas must become a full, new class.

In [ ]:
rxrycx = MultiQubitAnsatz(num_qubits, single_qubit_gate=['rx', 'ry'], two_qubit_gate='cx', depth=4)

## Adding and inserting layers

The circuits that current variational forms build are not extensible. For algorithms, such as adaptive VQE, adding new layers to the variational forms is necessary. 

Ideally, this could look like:

In [ ]:
varform.add(additional_layer, qubit_indices)

Other algorithms require the insertion of a new gate or layer not at the end of the variational form but somewhere in the middle.

In [ ]:
varform.add(additional_gate, qubit_indices, position)  # maybe also called varform.insert

For some applications, such as gradients, it is necessary to insert new gates at specific positions. These positions are characterised by a parameter `Parameter('θ')` of a gate.

In [ ]:
target_param = Parameter('θ')  # this variable is also used in the circuit 
varform.insert(target_param, instr_to_insert, qubit_indices)

## Combining variational forms

A variational form is essentially an instruction preparing a parameterized state. For chemistry, a this could be the preparation of certain excitation. For multiple excitations, we should be able to just add the respective variational forms.

In [ ]:
S0, S1 = single_excitation('θ0'), single_excitation('θ1')
D = double_excitation('θ2')
UCCSD = S0 + S1 + D

If some variational forms prepare subsystems, they should be able to be combined at certain qubit indices.

In [ ]:
varform = sub_varform.combine(other_varform, indices)

## DIY: special variational forms

Variational forms can be very diverse and for special problems special structures may perform better than others. Therefore, users should be able to specify their very own rotation and entanglement layers.

For constant layers, this could be:

In [ ]:
varform = Ansatz([my_rotation_layer, my_entanglement_layer], repeat=3, final_layer=my_rotation_layer)

If the layers differ per block, we might pass a function for the layers, which accepts a `block_num` argument.

In [ ]:
def special_entanglement(num_qubits, block_num):
    # returns the entanglement structure in block `block_num`
    
varform = Ansatz([my_rotation_layer, special_entanglement], repeat=5)

But to keep it as general as possible, it should be possible to wrap an arbitrary parameterized instructions as Ansatz or variational form, so it is accepted by the Aqua algorithms.

In [ ]:
varform = Ansatz(my_special_instruction)

## Feature map? Isn't that also a variational form?

The concept of a variational form is broad. To support a clear structure, we should group similar concepts. Then,
* users are less confused and see what concepts have the same base
* we can reuse the same base functionalities
* ensure that similar objects in Aqua behave the same way

Therefore, a common base for feature maps, variational forms (and others? `InitialState`? `UncertaintyModel`?) would be sensible.

In [ ]:
class FeatureMap(Ansatz):
    # implement
    
class MultiQubitVarform(Ansatz):
    # implement

class InitialState(Ansatz): 
    # implement
    
class UncertaintyModel(Ansatz):
    # implement

## Lazy parameterization

Terra provides the `Parameter` class for parameterized circuits. We should make use of this and allow for an easier construction of circuits, without requiring what we don't yet need: Parameters. Parameters are only necessary once a circuit is executed, not already when the variational form is constructed.

In [ ]:
circuit = varform.construct_circuit()  # no argument!

Maybe also:

In [ ]:
circuit = varform.construct_circuit(parameter_prefix='θ')

## Optional barriers

Barriers can be very useful for visualization, but if a circuit is executed only and not looked at, we don't need them. Why insert them by default?

In [ ]:
circuit = varform.construct_circuit(insert_barriers=True)  # False by default

# User benefit<a id='benefit'></a>

1. The ''frontend'' user
    1. can get creative and easily assemble new variational forms
    2. can `construct_circuit` without specifying parameters, and thereby benefits from potential speedup without even touching Terra's `Parameter` class
2. The researcher
    1. needs less copy+paste to implement standard variational forms
    2. can also benefit from `Parameter` related speedup
    3. has lots of lots of flexibility
        * by adding and combining varforms
        * inserting new layers
        * wrapping parameterized instructions
    4. gets less confused by the multitude of Ansatz-like objects in Aqua

# Proposed Structure 

The Ansatz class reconciles all classes that prepare an ansatz or wavefunction. Objects of the Ansatz type share the Ansatz interface and can be used as building blocks in larger circuits and algorithms. This has the key advantage, that same concepts behave similar and we can avoid duplicate code. 

More specialized types, that can also provide an easy setup or special behaviour, are derived from the Ansatz. 
However, instead of introducing simply a new layer below `VariationalForm` and `FeatureMap` the Ansatz can simply replace these. 

See the next Figure.

![Ansatz design](ansatz_scheme.jpg)

## Minimizing the number of layers

Currently (Aqua 6.x) distinguishes between a `VariationalForm` and a `FeatureMap`, which are both abstract classes that specify interfaces. These interfaces, however, almost identical and could be merged. Further, to new users the terms variational form and feature map might be confusing, whereas the word ''Ansatz'' wraps both concepts.

Thus, by introducing the `Ansatz` as joint class and directly derive `PauliExpansion` and special variational forms from it we group similar concepts without introducing new levels of inheritance, which might be sources of confusion.

## Composition vs. Inheritance

The objects `RY`/`RYRZ` as well as `FirstOrderExpansion`/`SecondOrderExpansion` are displayed as objects derived from not `Ansatz` but an intermediate type. This is discussed in the [implementation section](#standard_varforms).

# Implementation <a id='implementation'></a>

To be discussed in more detail.

1. Ansatz
1. Variational forms
    1. TwoLocalGateAnsatz
    1. Standard Forms
1. Feature maps
    1. Pauli expansions
    1. First/Second/... order
1. Initial state objects

## Ansatz <a id='ansatz'></a>

### Requirements recap

1. Take a list of instructions and a number of repetitions and construct an Ansatz of desired depth
2. Provide `construct_circuit(params=None, insert_barriers=False)`. Note, that `params` must support `list[float | Parameter]`.
3. Combine two Ansatz objects at given indices
4. Insert new instructions at a specified place
5. Backward compatibility with all variational form objects

Code checklist:

In [ ]:
def __init__(self, layers, repeat, final_layer)

def construct_circuit(params=None, parameter_prefix=None, insert_barriers=False)

def __add__(other)  # for Ansatz + Ansatz

def combine(other, connection_indices=None)  # for Ansatz + Ansatz at special indices

def add(instruction, indices=None, position=end)  # to add a new instruction

def insert_at_param(target_param, instruction, indices=None)   # to insert after/before a certain parameter

### Implementation proposal #1: `QuantumCircuit`

Make an Ansatz essentially a `QuantumCircuit` object. 

#### Advantages 

* already provides: `__add__(QuantumCircuit/Ansatz)` and `append(Instruction)`

#### Technical Issues

* cannot add two circuits of different sizes, nor combine them on specified indices
* `append` can only add an `Instruction` to the end of the circuit
* a circuit doesn't have `construct_circuit` (obviously)
* parameter setting by `list[float | Parameter]` not inherently supported
* how to insert barriers or new gates at arbitray positions? (use `data` member?)

#### Design Issues

* inserts another level of inheritance (or maybe just use `QuantumCircuit` as component?)
* `construct_circuit` is required for backward compatibility and the method name is very misleading 
* an Ansatz fits more the concept of an Instruction in the sense that it has no registers attached. Therefore it should probably not be a `QuantumCircuit`

### Implementation proposal #2: `Gate`

Make an Ansatz essentially a `Gate` object.

#### Advantages

* intuitively that's what an Ansatz is: a (parameterized) gate
* provides some required functionalities: allows setting of parameters as list, can be controlled, repeated, etc. 
* no registers bound to it
* stores parameters as `list`, setting parameters by `list[float | Parameter]` is straightforward
* a `construct_circuit` method makes sense and is maybe missing anyways (or `to_circuit`)

#### Technical Issues

* how to insert barriers or new gates at arbitrary positions?
* need to implement the appending of new gates/instructions (use `definition`?)

#### Design Issues

* inserts another level of inheritance (or maybe just use `Gate` as component?)

### Implementation proposal #3: `list[Gate]`

Base an Ansatz on a list of `Gate`s or `Instruction`s (each item with a set of qubit indices).

#### Advantages

* natural implementation of repeating layers
* appending new instructions is trivial
* combining two Ansaetze is straightforward (just merge the Instruction lists)
* design fits the expectation of a variational form and feature map: repeated layers of instructions
* barriers are easily inserted in between the layers

#### Technical Issues

* same issue as with `Gate`s to insert a new instruction somewhere inside a layer

#### Design Issues

* ?

### Conclusion

#1 (`QuantumCircuit`) is probably not suitable, compared to the advantages of #2 (`Gate`) and #3 (`list[Gate]`).

### Pseudo-code: #2 (`Gate`)

#### To-do

* `num_qubits`

In [ ]:
class Ansatz(Gate):
    def __init__(self, layers, repeat, final_layer):
        # repeat layers and add to self.definition

    def construct_circuit(params=None, parameter_prefix=default_prefix, insert_barriers=False):
        circuit = QuantumCircuit(self.num_qubits)
        # if insert_barriers: iterate over self.definition and add barriers in between layers
        # else: just append self to circuit
        # set parameters (and create them according to parameter_prefix, if none are given)
        return circuit
    
    def combine(other, connection_indices=None):  # for Ansatz + Ansatz at special indices
        # if connection_indices are given, adjust the qargs of other
        # merge the definitions of self and other to a third Ansatz 
        # ensure other attributes are properly set and return
    
    def __add__(other):  # for Ansatz + Ansatz
        return self.combine(other)
    
    def add(instruction, indices=None, position=end):  # to add a new instruction
        # insert (instruction, indices) at position at self.definition

    def insert_at_param(target_param, instruction, indices=None):   # to insert after/before a certain parameter
        # TODO

#### Conclusion

Suitable. 

#### Problems

* How to implement `insert_at_param`? 

### Pseudo-code: #3 (`list[Gate]`)

In [ ]:
class Ansatz():
    def __init__(self, layers, repeat, final_layer):
        # repeat layers and add to self._layers
        self._layers = []

    def construct_circuit(params=None, parameter_prefix=default_prefix, insert_barriers=False):
        circuit = QuantumCircuit(self.num_qubits)
        for layer in self._layers:
            # append to circuit, insert_barrier if necessary
            
        # set parameters (and create them according to parameter_prefix, if none are given)
        return circuit
    
    def combine(other, connection_indices=None):  # for Ansatz + Ansatz at special indices
        # if connection_indices are given, adjust the qargs of other
        # merge the definitions of self and other to a third Ansatz 
        # ensure other attributes are properly set and return
    
    def __add__(other):  # for Ansatz + Ansatz
        return self.combine(other)
    
    def add(instruction, indices=None, position=end):  # to add a new instruction
        # insert (instruction, indices) at position at self.definition

    def insert_at_param(target_param, instruction, indices=None):   # to insert after/before a certain parameter
        # TODO

#### Conclusion

This is essentially a Instruction wrapping Instructions. But since an Instruction can already contain others, we can simply stick with Scenario #2. 

More detail: If we add all instructions in the initializer `layers` to the definition of the `Gate`, thats equivalent to keeping a list of these instructions. Introducing `Ansatz` in this way is as obsolete the `InstructionSet` in Terra. 

#### Problems

* How to implement `insert_at_param`? 

## Result

Slight preference for design #2: The Ansatz as Gate, i.e. `class Ansatz(Gate)`.

Reasons:
* conceptually the most sensible
* some functionality exists already
* doesn't rebuild the deprecated `InstructionSet`

However design #3, where the Ansatz holds a list of gates, also is a very suitable design.

## Variational forms <a id='varforms'></a>

![](img/ansatz_varform_scheme.pdf)

### Backward compatibility

Does Ansatz provide all functionalities of the current `VariationalForm`? 

Almost.

Missing:
* `setting(self)`
* `support_parameterized_circuit(self)`
* `parameter_bounds(self)`
* `depth`

#### `setting`

Prints the `CONFIGURATION` plus `self.__dict__`. Is this necessary? 

If yes, this is easily added to the `Ansatz` and might make sense for debug purposes.

#### `support_parameterized_circuit`

This is true when [ask kevin when exactly]. If the transpiler cannot check this automatically, this can be added to Ansatz. Either let the user set it, or check all gates for compatibility and set automatically.

#### `parameter_bounds`

The `VQAlgorithm` expects a variational form to have this parameter. The bounds are then used for the optimization. Since not all optimizers however accept bounds this raises a design issue: Setting parameter bounds for a variational form suggests, that the variational form will enforce these bounds. That's not correct. 

A more correct behaviour would be to feed the parameter bounds to the optimizer directly and remove this member from the variational forms. This is especially true since the `VQAlgorithm` is only one application of variational forms and they are much more widely used.

Thus, this member should not be added to `Ansatz` but this requires changes in the `VQAlgorithm`.

#### `depth`

This equals the `repeat` keyword of `Ansatz`. Maybe we should rename `repeat` to `depth`. In favour of renaming: the `FeatureMap`s also take a `depth` keyword.

Hence, rename `repeat` to `depth` and implement a corresponding property.

## TwoLocalGateAnsatz

### Requirement recap

Input: number of qubits, one qubit gate(s), two qubit gate(s), depth, entanglement, skip_unentangled_qubits, skip_final_rotation, (initial state?)

Output: the variational form

### Implementation proposal

Use the initializer to set up the rotation and entanglement layers, provided with the single- and two-qubit gates. Incorporate functionalities to set the entangler map. 

### Pseudo-code

In [ ]:
class TwoLocalGateAnsatz(Ansatz):
    def __init__(self, num_qubits, single_qubit_gate=None, two_qubit_gate=None, depth=default_depth,
                 entanglement=default_entanglement, initial_state=None):
        # verify single and two qubit gates, probably use static method
        for block_number in range(depth):
            # use self.add() with get_rotation_layer() and get_entanglement_layer() to add the layers
        # add final layer
        
    def _get_rotation_layer(self, block_number):
        # return the rotation layer as instruction
        
    def _get_entanglement_layer(self, block_number):
        # return the entanglement layer as instruction
    
    @property
    def single_qubit_gate(self):  # same for two_qubit_gate
        # return single qubit gate
        
    @staticmethod
    def verify_gate(self, gate, kind='single'):
        # if gate is str, return gate, otherwise check that single or two qubit gate
        
    def get_entangler_map(self, block_number=0):
        # use self._entanglement and block_number to return the correct entangler_map

#### Potential issues

* `get_entangler_map` is no static method anymore
* `validate_entangler_map` doens't exist anymore

#### Points to discuss

* Should the keywords `entanglement` and `entangler_map` be merged to `entanglement`?
* Is the `initial_state` keyword necessary?

Should the keywords `entanglement` and `entangler_map` be merged to `entanglement`?

#### Pro

* The user doesn't need to think about which keyword he needs to use for the entanglement input, whether it is specified via string or pairs of qubits.
* Assume we add a new possible input type, e.g. `Callable`. Then we would require yet another keyword and choose which keyword is actually active if multiple are set. With one keyword only, we just have to add an `isinstance` if-statement (which we anyways need).
* Having multiple keywords forces the code to decide which keyword overrules which. This can be opaque to the user. If we don't enforce a "keyword hierarchy", we could throw errors if multiple are set. And nobody likes runtime errors (see PEP [which number again?]). 

#### Contra

* The signature doesn't clearly show what input types of entanglement exist. (Then again, there is such a thing as documentation strings.)

**Conclusion** Yes, merge `entanglement` and `entangler_map` in `entanglement`.

## Standard variational forms<a id='standard_varforms'></a>

Such forms as `RY` or `RYRZ`.

### Implementation proposal #1

As subclass of `TwoLocalGateAnsatz`.

#### Advantages

* super easy to implement
* natural: specification of two local gate ansatz by fixing the single qubit gate and providing a default two qubit gate

#### Issues

* adds another level of inheritance
* `RY(Ansatz)` is elegant to read, since `RY` is just an Ansatz

### Implementation proposal #2

As subclass of `Ansatz`, using `TwoLocalGateAnsatz` as component.

#### Advantages

* `RY(Ansatz)` is intuitive to read
* no new level of inheritance

#### Issues

* probably need to wrap all functions of `TwoLocalGateAnsatz`

### Conclusion

Inheritance is easier to implement and reading `RY(TwoLocalGateAnsatz)` is still comprehensible.
Therefore, choose inheritance over composition.

Or is there an easy way for using composition, that we missed?

### Pseudo-code

In [ ]:
class RY(TwoLocalGateAnsatz):
    def __init__(self, ...):  # repeat all keywords here to enable IDE detection
        super().__init__(single_qubit_gate='ry', ...)

In [ ]:
class RYRZ(TwoLocalGateAnsatz):
    def __init__(self, ...):
        super().__init__(single_qubit_gate=['ry', 'rz'], ...)

## Feature maps <a id='featmaps'></a>

![](img/ansatz_featmap_scheme.pdf)

### Backward compatibility

Does Ansatz provide all functionalities of the current `FeatureMap`? 

Yes -- except `self.feature_dimension`.

#### `feature_dimension`

Possible solutions include

* New intermediate class called `FeatureMap(Ansatz)`: An additional class for just one new method, that exists under another name, is not really justified.

* Add the method `feature_dimension` to `Ansatz`: If Ansatz used as variational form this method remains unused.

* Deprecate `feature_dimension`, use `num_parameters` instead: Deprecating `feature_dimension` is probably not a good idea, since it is a legitimate attribute of a feature map.

**Conclusion** The best solution seems to be adding the `feature_dimension` method to `Ansatz`.

### Implementation proposal

The hierarchy within the feature map branch should be the same as for variational forms. Meaning, we implement a `PauliExpression` and the specialized classes as derived objects. This design is also the current form.

However -- is the additional layer `PauliZExpansion` truly necessary?

In [ ]:
def __init__(self, feature_dimension, depth=2, entangler_map=None, entanglement='full', z_order=2, 
            data_map_func=self_product):
    self.validate(locals())
    pauli_string = []
    for i in range(1, z_order + 1):
        pauli_string.append('Z' * i)
    super().__init__(feature_dimension, depth, entangler_map, entanglement, paulis=pauli_string, 
                     data_map_func=data_map_func)

#### Pro

* simplifies the definition of Pauli Z expansion feature maps (such as `First/SecondOrderExpansion`)

#### Contra

* introduces an additional layer of inheritance
* if `First/SecondOrderExpansion` is the main motivation for this class, then the two minor changes they introduce is probably not worth introducing the `PauliZExpansion`